import libraries

In [3]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm
from functools import reduce
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
from itertools import product

import custom functions

In [ ]:
# Specify the full path to the module (including the filename)
module_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/parser.py'

# Import the module using importlib (a workaround if the standard import doesn't work)
import importlib.util
spec = importlib.util.spec_from_file_location("parser", module_path)
parser = importlib.util.module_from_spec(spec)
spec.loader.exec_module(parser)


FileNotFoundError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1 Data collection

## 1.1. Get the list of master's degree courses

In [ ]:
def extract_masters(this_url):
    result_url = requests.get(this_url)
    result_soup = BeautifulSoup(result_url.text, 'html.parser')
    result_links = result_soup.find_all('a', {'class': 'courseLink'})
    result_list = []
    for item in result_links:
        result_list.append(item['href'])
    return result_list

In [ ]:
# Output file path
output_file_path = 'msc_urls.txt'
# Loop through the first 400 pages and write results to the output file
with open(output_file_path, 'a') as output_file:
    for page_number in range(1, 401):
        page_url = f'https://www.findamasters.com/masters-degrees/msc-degrees/?PG={page_number}'
        page_results = extract_masters(page_url)
        for url in page_results:
            output_file.write(f'{url}\n')
        time.sleep(1)

In [ ]:
with open(output_file_path, 'r') as file:
    lines = file.readlines()
    number_of_lines = len(lines)

print(f'The file {output_file_path} contains {number_of_lines} rows.')

The file msc_urls.txt contains 6000 rows.


## 1.2. Crawl master's degree pages

In [ ]:
# Function to download HTML from a URL with prefix and save it to a file
def download_and_save_html(url, output_path):
    full_url = 'https://www.findamasters.com/' + url
    try:
        response = requests.get(full_url)
        if response.status_code == 200:
            with open(output_path, 'w', encoding='utf-8') as html_file:
                html_file.write(response.text)
        else:
            print(f"Failed to download {full_url}. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {full_url}: {e}")

# Function to create a directory if it doesn't exist
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

# Main directory for downloaded HTML files
main_directory = 'downloaded_pages'
create_directory(main_directory)

# Iterate through the URLs and download the HTML
with open('msc_urls.txt', 'r') as file:
    for index, url in enumerate(file, start=1):
        # Remove whitespaces and newline characters from the URL
        url = url.strip()

        # Generate the directory path for the current page
        page_directory = os.path.join(main_directory, f'page_{(index - 1) // 15 + 1}')
        create_directory(page_directory)

        # Generate the output HTML file path
        output_path = os.path.join(page_directory, f'course_{index}.html')

        # Download the HTML and save it to the file
        download_and_save_html(url, output_path)
        time.sleep(3)

print("Download complete for all pages.")

Download complete for all pages.


#[1.3] Parse downloaded pages



At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format as desiried.

#Access the stored HTML pages

In [ ]:
%pip install google-colab-shell
# import the module once
from google_colab_shell import getshell

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=80e086c86d11d45d2f29c9a491589816fae1262d0347b13dc5ea33ecab08cf12
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell


used command

---mettere modalità python--
git clone https://github.com/marinazanoni/nome_provvisorio.git

after accesing the repository

In [ ]:
def parser(html_page):

    # Define your default values here
    default_values = {
    'courseName': None,
    'universityName': None,
    'facultyName': None,
    'isItFullTime': None,
    'description': None,
    'startDate': None,
    'fees': None,
    'modality': None,
    'duration': None,
    'city': None,
    'administration': None,
    'country': None,
    'url': None
}

    # Create a defaultdict with default values
    info = defaultdict(lambda: default_values)

   # Extract the text (HTML)
    with open(html_page, 'r', encoding='utf-8') as file:
        html_content = file.read()
    page_soup = BeautifulSoup(html_content, 'html.parser')

    # COURSE NAME
    page_links = page_soup.find_all('h1', {'class':'text-white course-header__course-title'})
    if page_links:
        first_h1_text = page_links[0].text
        info['courseName'] = str(first_h1_text)
    #else:
        # Handle the case when no 'h1' with 'data-permutive-title' is found
    #    print("No course name found.")
    #courseName = str(first_h1_text)

    # UNIVERSITY NAME
    # Extracting our tag of interest for the Name
    page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
    # Extracting the name of the course as string and print it
    info['universityName'] = str(page_links[0].contents[0])

    # FACULTY NAME
    page_links = page_soup.find_all('a', {'class': 'course-header__department'})
    info['facultyName'] = str(page_links[0].contents[0])

    # FULL TIME
    page_links = page_soup.find_all('a', {'class': 'inheritFont'})
    info['isItFullTime'] = str(page_links[0].contents[0])

    # SHORT DESCRIPTION
    page_links = page_soup.find('div', {'id': 'Snippet'})
    info['description'] = str(page_links.get_text(separator='\n').strip())

    # STARTING DATE
    page_links = page_soup.find('span', {'class': 'key-info__start-date'})
    info['startDate'] = str(page_links.get_text())

    # FEES
    page_links = page_soup.find('a', {'class': 'noWrap inheritFont'})
    page_links = page_soup.find('div', {'class': 'course-sections__fees'})
    if page_links:
        fees_text = page_links.get_text(separator='\n').strip()
        # Remove "Fees" from the text content
        info['fees'] = fees_text.replace('Fees', '').strip()


    # MODALITY
    page_links = page_soup.find('span', {'class': 'key-info__content key-info__qualification py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
    # Get all elements within the span using find_all
    elementsWithinSpan = page_links.find_all('a')
    info['modality'] = ' '.join([element.text.strip() for element in elementsWithinSpan])

    # DURATION
    page_links = page_soup.find('span', {'class':'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
    info['duration']=str(page_links.text)

    # CITY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
    info['city']=str(page_links.text)

    # ADMINISTRATION
    page_links = page_soup.find('span', {'class':'course-header__online-flag badge bg-white p-2 h6 shadow-sm mr-1'})
    if page_links == None:
        info['administration'] = "On Campus"
    else:
        info['administration']=str(page_links.text)

    # COUNTRY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
    info['country']=page_links.text

    #URL
    page_links = page_soup.find('link')
    info['url'] = page_links.get('href')


    return(pd.DataFrame([info]))


*Searching for a tipical hidden file and delete it in order not to have it when exctracting the information from the folder*

In [ ]:
parser('/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_1/course_1.html')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
file_path_to_remove = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/.DS_Store'
# Delete the hidden file
os.remove(file_path_to_remove)
print(f"The file {file_path_to_remove} has been deleted.")

In [ ]:
len(parsed_dfs)

5979

*Applying the function to our html pages*

In [ ]:
# Specifing the path of the folder
folder_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages'

# List all files in the folder
files = os.listdir(folder_path)

parsed_dfs = []

# Iterate through all folders and subfolders using os.walk
for folder_path, _, file_names in os.walk(folder_path):
    # Check if there are files in the current folder
    if file_names:
        # Iterate through each file in the current folder
        for file_name in file_names:
            file_path = os.path.join(folder_path, file_name)

            # Store the information only is the dictionary is not empty (has at list a name course)
            try:
                # Parse the file and append the result to the list
                parsed_df = parser(file_path)
                parsed_dfs.append(parsed_df)
            except Exception as e:
                # Print the file path when an exception occurs
                print(f"Error parsing file: {file_path}")
                # print(f"Error details: {e}")

# Concatenate all DataFrames in the list
concatenated_df = pd.concat(parsed_dfs, ignore_index=True)


Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_118/course_1765.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_119/course_1772.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_128/course_1910.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_140/course_2086.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2929.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2931.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_215/course_3213.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_291/course_4357.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_293/course_4395.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_2

In [ ]:
concatenated_df['url'][3]

'https://www.findamasters.com/masters-degrees/course/applied-economics-banking-and-financial-markets-online-msc/?i280d8352c56675'

In [ ]:
# Display the concatenated DataFrame
concatenated_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
concatenated_df.shape

(5979, 13)

In [ ]:
print(6000-5979, 'were not valid pages')

21 were not valid pages


In [ ]:
parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   courseName      1 non-null      object
 1   universityName  1 non-null      object
 2   facultyName     1 non-null      object
 3   isItFullTime    1 non-null      object
 4   description     1 non-null      object
 5   startDate       1 non-null      object
 6   fees            1 non-null      object
 7   modality        1 non-null      object
 8   duration        1 non-null      object
 9   city            1 non-null      object
 10  administration  1 non-null      object
 11  country         1 non-null      object
 12  url             1 non-null      object
dtypes: object(13)
memory usage: 232.0+ bytes


In [ ]:
# Specify the path where you want to save the .tsv file
tsv_file_path = '/content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv'

# Save the DataFrame to a .tsv file
concatenated_df.to_csv(tsv_file_path, sep='\t', index=False)

print(f".tsv file saved at: {tsv_file_path}")

.tsv file saved at: /content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv


### *saving singularly the information about each master (html page)*

In [ ]:
# Iterate through each DataFrame in the list
for index, parsed_df in enumerate(parsed_dfs):
    # Iterate through each row in the DataFrame
    for row_index, row in parsed_df.iterrows():
        # Replace NaN values with empty strings
        single_row = row.fillna(' ')
        # Specify the path where you want to save the .tsv file for the current row
        tsv_file_path = f'/content/drive/MyDrive/HM3-ADM/HW3_ADM/Courses/course_{index}.tsv'
        # Save the single row DataFrame to a .tsv file
        with open(tsv_file_path, 'w') as file:
            file.write('\t'.join(single_row))

#Preproccessing

Uploading the file created after parsing the information

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv',sep='\t')
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Linguistics - MSc,University of Glasgow,College of Arts & Humanities,Full time,This Masters focuses on how linguistic researc...,September,Please see the university website for further ...,MSc,1 year full-time; 2 years part-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import *

# Download the stopwords dataset if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#[2.0.0]

## Stemming

In [ ]:
# Make a copy of the original DataFrame
processed_df = df.copy()

# stemmer
stemmer = PorterStemmer()

processed_df['description'] = processed_df.description.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.split(' ')]))

##Lower, removing punctuation and stepwords

In [ ]:
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and punctuation from a text
def clean(text):
    """
    The following function returns the filtered element for each column of a dataframe.
    Filtering operation consists in removing punctuation and removing stopwords given text with lower case
    """
    words = word_tokenize(str(text))
    # Remove punctuation using NLTK and string.punctuation
    filtered_words = [word for word in words if word not in string.punctuation + "'’...?'+,-'‘“”„•…›✓"]
    # Remove stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Selecting columns
text_columns = processed_df.select_dtypes(include='object').columns
# Lower
processed_df[text_columns] = processed_df[text_columns].apply(lambda x: x.str.lower() if x.dtype == 'O' else x)

# Exclude fees from removing punctuation
exclude_column = 'fees'

# Applying clean function to all the valid columns except the excluded one
processed_df[text_columns.difference([exclude_column])] = processed_df[text_columns.difference([exclude_column])].applymap(clean)


we verify which element we're filtering to check and think if something else need to be considered. Checking buy some outuput of the stemmed words we see some additional punctuation we don't want to consider which was added within the clean function.

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

We can know appreciate the differences of the two datasets (before and after preprocessing). Of our interesting will be the columns **fees** and **description**.

In [ ]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,air quality solutions msc,university leeds,institute transport studies,full time,7 million peopl estim die everi year due harm ...,september,"uk: £12,500 (total) \ninternational: £28,750 (...",msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
1,analytical toxicology msc,king college london,faculty life sciences medicine,full time,analyt toxicolog msc uniqu studi cours integr ...,see course,please see the university website for further ...,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
2,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,comput scienc foundat mani excit cutting-edg t...,september january,please see the university website for further ...,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
3,applied economics banking financial markets on...,university bath,university bath online,part time,polit uncertainti financ recruit demands econo...,september january,cost per 10 credits £722* (10% alumni discount...,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...
4,applied linguistics msc,university glasgow,college arts humanities,full time,thi master focus linguist research help solv l...,september,please see the university website for further ...,msc,1 year full-time 2 years part-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...


#[2.0.1]

 ## we want the field fees to collect numeric information


We consider only *fees* column and deepen what we find

In [ ]:
raw_fees= pd.DataFrame(processed_df['fees'])

In [ ]:
raw_fees.head()

,fees,new_fees,new_fees_after
0,"uk: £12,500 (total) \ninternational: £28,750 (...",NaN,None
1,please see the university website for further ...,NaN,None
2,please see the university website for further ...,NaN,None
3,cost per 10 credits £722* (10% alumni discount...,NaN,None
4,please see the university website for further ...,NaN,None


In [ ]:
# Missing values for fees
print(sum(raw_fees['fees'].isna()==True))

122


we don't want to do anything special with these none value and instead we probably want to set other values of the column to None, for examples those containing only charathers which explains were to get the correct information about *fees*

In [ ]:
# Filter rows where 'fees' contains at least one digit
raw_fees_filtered = raw_fees[raw_fees['fees'].str.contains(r'\d', na=False, regex=True)]

In [ ]:
pattern = r'(?P<symbol>[^\d\s])(?P<value>\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'

In [ ]:
raw_fees_filtered[raw_fees_filtered['fees'].str.contains(r'\d{4}?(-\d{4})') & raw_fees_filtered['fees'].str.contains(pattern)]['fees'].loc[30]

<ipython-input-154-77492536b51b>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  raw_fees_filtered[raw_fees_filtered['fees'].str.contains(r'\d{4}?(-\d{4})') & raw_fees_filtered['fees'].str.contains(pattern)]['fees'].loc[30]


'tuition fees (2024-2025): \neu/eea nationals: 1092 €/year \nnon eu/eea nationals:2500 €/year'

there are lot of missings data or more complex information we need to filter. To filter we follow the given guidline:


> in case of multiple information, retrieve only the highest fees. This suits perfectly the following example
>>*'uk £13,000 total international £29,000 total',
       '*

>In this other case will still apply, altough we're losing information so we will end up overestimating or underestimating the price. We decide like not to drop these value as can theorically give a (loose) range. Some general garbage are typically known when wanting to buy a master


>>*'eu fee course €10,820 €5,475 year 1 part-time €5,475 year 2 part-time cipd subscription fee €190 included student fee. non-eu fee course €18,700 includes cipd subscription fee €190 paid student',*


In [ ]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY'
    # Add more currency symbols as needed
}

# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency(price, target_currency='USD'):
    try:
        # Extract the currency symbol from the string
        currency_symbol = price[0]

        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(price[1:].replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)
        #return currency_symbol

    except Exception as e:
       return None


In [ ]:
pattern = r'(?P<symbol>[^\d\s])(?P<value>\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'

def return_cost(stri):
    """
    This function returns a string built up by the currency and
    the maximum number it appears in the string given in input
    """
    # Check if the input is a string
    if not isinstance(stri, str):
        return None  # Return None if the input is not a string

    # Search for the patterns in the string (currency, values)
    match = re.findall(pattern, stri)

    if not match:
        return None  # Return None if no currency symbols and values are found

    # Store all values
    result = [f"{currency[0]}{currency[1].replace(',', '')}" for currency in match]

    # Convert to a common currency format
    converted_list = [convert_to_common_currency(elem) for elem in result]

    # Exclude None values before finding the maximum
    non_none_values = [value for value in converted_list if value is not None]

    if not non_none_values:
       return None  # Return None if all values are None

    # Extract the maximum
    max_v = max(non_none_values)
    return max_v

text_columns = processed_df.select_dtypes(include='object').columns

# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['new_fees'] = raw_fees['fees'].apply(lambda x: return_cost(x) if x is not None else None)


In [ ]:
pattern1 = r'(?P<symbol>[^\d\s])(?P<value>\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'

def return_cost(stri):
    """
    This function returns a string built up by the currency and
    the maximum number it appears in the string given in input
    """
    # Check if the input is a string
    if not isinstance(stri, str):
        return None  # Return None if the input is not a string

    # Search for the patterns in the string (currency, values)
    match = re.findall(pattern, stri)

    if not match:
        return None  # Return None if no currency symbols and values are found

    # Store all values
    result = [f"{currency[0]}{currency[1].replace(',', '')}" for currency in match]

    # Convert to a common currency format
    converted_list = [convert_to_common_currency(elem) for elem in result]

    # Exclude None values before finding the maximum
    non_none_values = [value for value in converted_list if value is not None]

    if not non_none_values:
       return None  # Return None if all values are None

    # Extract the maximum
    max_v = max(non_none_values)
    return max_v

text_columns = processed_df.select_dtypes(include='object').columns

# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['new_fees'] = raw_fees['fees'].apply(lambda x: return_cost(x) if x is not None else None)


fees were currency is after the values

In [ ]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY'#,
   # 'sek':'SEK', #swedish corona
    #'euro':'EUR',
    #'hkn' : 'HNK',
    #'euros': 'EUR',
    #'jpy': 'JPY',
    #'hkd':'HDK',
    #'isk':'ISK'
}

# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency1(price, target_currency='USD',currency_symbol = '£',amount=0):
    try:
        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(amount.replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)
        #return currency_symbol

    except Exception as e:
       return None


In [ ]:
pattern1 = r'(?P<symbol>[^\d\s$£€]?)(?P<value>\d{1,8}(?:,\d{3})*(?:\.\d{2})?)(?P<symbol_after>\s?[^\d\s])'
pattern2 =r'(?P<symbol>[^\d\s]?)(?P<value>\d{1,8}(?:,\d{3})*(?:\.\d{2})?)(?P<symbol_after>\s?[^\d\s])'
#currency_symbols = '|'.join(re.escape(symbol) for symbol in currency_mapping.keys())
#pattern3 = r'(?P<symbol>(?:{}))[^\d\s]?(?P<value>\d{1,8}(?:,\d{3})*(?:\.\d{2})?)[^\d\s]?(?P<symbol_after>\s?[^\d\s])'.format(currency_symbols)


def return_cost(stri):
    """
    This function returns a string built up by the currency and
    the maximum number it appears in the string given in input
    """
    # Check if the input is a string
    #print(stri)
    if not isinstance(stri, str):
        return None  # Return None if the input is not a string

    # Search for the patterns in the string (currency, values)
    match = re.findall(pattern1, stri)
#    print('inizio',match)

    # If currency is befor the value
    a = [value.strip()+symbol_after.strip('()') for _, value, symbol_after in match if value and symbol_after]
    b = [elem for elem in a if re.search(r'\d\s([£$€])',elem)]
    # Convert to a common currency format
    converted_list = [convert_to_common_currency1(elem,elem[-1],amount=elem[:-2]) for elem in b]
  #  print(converted_list)
    # Exclude None values before finding the maximum
    non_none_values = [value for value in converted_list if value is not None]
    if not non_none_values:
        return None  # Return None if all values are None

    # Extract the maximum
    max_v = max(non_none_values)
    if max_v>0:
        return max_v


def return_cost2(stri):
    if not isinstance(stri, str):
        return None  # Return None if the input is not a string

    match = re.findall(pattern2, stri)
   # print('fine',match)
    c = [symbol.strip()+value.strip() for symbol, value,_ in match if symbol and value]
  #  print(c)
    # Convert to a common currency format
    for elem in c:
        if elem[0] in(['£', '$', '€']):
            converted_list1 = [convert_to_common_currency1(elem,elem[0],amount= elem[1:].replace(',', '')) for elem in c]
           # print(converted_list1)
            # Exclude None values before finding the maximum
            non_none_values1 = [value for value in converted_list1 if value is not None]

            if not non_none_values1:
                return None  # Return None if all values are None

            # Extract the maximum
            max_v1 = max(non_none_values1)
            if max_v1>0:
                return max_v1
    return None


text_columns = processed_df.select_dtypes(include='object').columns

# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['new_fees_after'] = raw_fees['fees'].apply(lambda x: return_cost(x) if x is not None else None)

# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['new_fees'] = raw_fees['fees'].apply(lambda x: return_cost2(x) if x is not None else None)

In [ ]:
raw_fees.head()

,fees,new_fees,new_fees_after
0,"uk: £12,500 (total) \ninternational: £28,750 (...",35768.85,NaN
1,please see the university website for further ...,NaN,NaN
2,please see the university website for further ...,NaN,NaN
3,cost per 10 credits £722* (10% alumni discount...,898.26,NaN
4,please see the university website for further ...,NaN,NaN


In [ ]:
len(raw_fees[~raw_fees[['new_fees', 'new_fees_after']].isna().all(axis=1)])


1101

In [ ]:
pattern2 = r'(?P<symbol>[^\d\s]?)(?P<value>\d{1,8}(?:,\d{3})*(?:\.\d{2})?)(?P<symbol_after>\s?[^\d\s])'
#pattern2 = r'(?P<symbol>[^\d\s$£€]?)(?P<value>\d{1,8}})'


def return_cost2(stri):
    """
    This function returns a string built up by the currency and
    the maximum number it appears in the string given in input
    """
    # Check if the input is a string
    #print(stri)
    if not isinstance(stri, str):
        return None  # Return None if the input is not a string

    # Search for the patterns in the string (currency, values)
    match = re.findall(pattern1, stri)
#    print('inizio',match)

    # If currency is befor the value
    a = [value.strip()+symbol_after.strip('()') for _, value, symbol_after in match if value and symbol_after]
    b = [elem for elem in a if re.search(r'\d\s([£$€])',elem)]
    # Convert to a common currency format
    converted_list = [convert_to_common_currency1(elem,elem[-1],amount=elem[:-2]) for elem in b]
    # Exclude None values before finding the maximum
    non_none_values = [value for value in converted_list if value is not None]
    if not non_none_values:
        return None  # Return None if all values are None

    # Extract the maximum
    max_v = max(non_none_values)
    if max_v>0:
        return max_v


text_columns = processed_df.select_dtypes(include='object').columns

# Return the list of currency symbols and costs encountered in 'fees' field
raw_fees['new_fees_after'] = raw_fees['fees'].apply(lambda x: return_cost2(x) if x is not None else None)


In [ ]:
len(raw_fees[~raw_fees['new_fees'].isna()])

53

###here we have filter fees column with only one cost each row, where available

In [ ]:
len(raw_fees)

53

In [ ]:
1095+53


1148

In [ ]:
raw_fees[~raw_fees[['new_fees', 'new_fees_after']].isna().all(axis=1)].index

Int64Index([   0,    3,    5,    9,   11,   14,   19,   28,   29,   30,
            ...
            5939, 5945, 5954, 5958, 5965, 5967, 5970, 5971, 5972, 5975],
           dtype='int64', length=1101)

In [ ]:
complementary_indices = list(set(raw_fees.index).difference(raw_fees[~raw_fees[['new_fees', 'new_fees_after']].isna().all(axis=1)].index))
complementary_df = raw_fees.loc[complementary_indices]

for index, row in complementary_df.iterrows():
    statement = row['fees']
    print(' -----------------------',statement,sep='\n')
   # print(index)

Output streaming troncato alle ultime 5000 righe.
please see the university website for further information on fees for this course.
 -----------------------
please see the university website for further information on fees for this course.
 -----------------------
please see the university website for further information on fees for this course.
 -----------------------
please see the university website for further information on fees for this course.
 -----------------------
please see the university website for further information on fees for this course.
 -----------------------
tuition fee, first payment: 72,900 sek 
 
tuition fee, total: 255,800 sek
 -----------------------
tuition fee, first payment: 77,500 sek 
 
tuition fee, total: 310,000 sek
 -----------------------
please see the university website for further information on fees for this course.
 -----------------------
please see the course page for further information.
 -----------------------
please see the university w

In [ ]:
raw_fees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5979 entries, 0 to 5978
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   fees    1049 non-null   float64
dtypes: float64(1)
memory usage: 46.8 KB


In [ ]:
1095/5979

0.18314099347717008

In [ ]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY',
    'eur': 'EUR'
    # Add more currency symbols as needed
}

# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency(row, target_currency='USD'):
    if pd.isna(row['fees']):
        return None
    try:
        # Extract the currency symbol from the string
        currency_symbol = row['fees'][0]

        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(row['fees'][1:].replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)

    except Exception as e:
       return None


In [ ]:
# Apply the conversion function to the 'fees' column
raw_fees['fees'] = raw_fees.apply(convert_to_common_currency, axis=1)

# Rename the column to indicate the common currency
common_currency_code = 'USD'
raw_fees.rename(columns={'fees': f'fees ({common_currency_code})'}, inplace=True)

In [ ]:
raw_fees.head()

,fees,new_fees
0,"uk: £12,500 (total) \ninternational: £28,750 (...",NaN
1,please see the university website for further ...,NaN
2,please see the university website for further ...,NaN
3,cost per 10 credits £722* (10% alumni discount...,NaN
4,please see the university website for further ...,NaN


sustituing and renaming the new column with the appropriate currance values

In [ ]:
processed_df['fees']= raw_fees['fees (USD)']
processed_df.rename(columns={'fees': 'fees (USD)'}, inplace=True)


In [ ]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,air quality solutions msc,university leeds,institute transport studies,full time,[nan],september,"uk £12,500 total international £28,750 total",msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
1,analytical toxicology msc,king college london,faculty life sciences medicine,full time,[nan],see course,please see university website information fees...,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
2,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,[nan],september january,please see university website information fees...,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
3,applied economics banking financial markets on...,university bath,university bath online,part time,[nan],september january,cost per 10 credits £722 10 alumni discount pa...,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...
4,applied linguistics msc,university glasgow,college arts humanities,full time,[nan],september,please see university website information fees...,msc,1 year full-time 2 years part-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...


#[2.1]

##[2.1.1]

In [ ]:
vocabulary = set()

In [ ]:
processed_df['description']= processed_df.description.apply(lambda row: word_tokenize(row))

In [ ]:
processed_df.description.head()

0    [7, million, peopl, estim, die, everi, year, d...
1    [analyt, toxicolog, msc, uniqu, studi, cours, ...
2    [comput, scienc, foundat, mani, excit, cutting...
3    [polit, uncertainti, financ, recruit, demands,...
4    [thi, master, focus, linguist, research, help,...
Name: description, dtype: object

In [ ]:
processed_df.description.apply(lambda row: [vocabulary.add(word) for word in row])


0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5974    [None, None, None, None, None, None, None, Non...
5975    [None, None, None, None, None, None, None, Non...
5976    [None, None, None, None, None, None, None, Non...
5977    [None, None, None, None, None, None, None, Non...
5978    [None, None, None, None, None, None, None, Non...
Name: description, Length: 5979, dtype: object

In [ ]:
vocabulary

{'mesc',
 'knowledge-bas',
 'dilemma',
 'engaged',
 'relationships',
 'two',
 'mini',
 'agro-',
 'breathe',
 'spear',
 'web-presence',
 'reorganis',
 'antique',
 'webst',
 'neurosensori',
 'nuanc',
 'deliverers',
 'statist',
 'high-tech',
 'dig',
 'stonehaven',
 'prototypes',
 'oncologist',
 'bankers',
 '96',
 'psychotherapist',
 'sharper',
 'inter-relationship',
 'non-democrat',
 'telecommunications',
 'characterized',
 'economists',
 'zooarchaeolog',
 'yet',
 'increases',
 'supporters',
 'nanoscale',
 'creator',
 'analyz',
 'crédit',
 'australian',
 'stakeholders.y',
 'ports',
 'consequ',
 'auf',
 'underli',
 'helmholtz-cent',
 'forensic/investig',
 'cables',
 'strathclyde',
 'inter-government',
 'bioscience',
 'twentieth',
 'superb',
 'appli',
 'project-work',
 'well-publicis',
 'commercialization',
 'got',
 'mainli',
 'conducted',
 'effect',
 'established',
 'alien',
 'however',
 'kick-start',
 'glastonbury',
 'signific',
 'thinking',
 'administrative',
 'decarbonisation',
 'comple

 Assign unique ID to each term in the vocabulary

In [ ]:
vocabulary_list = list(vocabulary)
vocabulary_dict = {word: index for index, word in enumerate(vocabulary_list)}
vocabulary_df=pd.DataFrame(list(vocabulary_dict.items()), columns=['Word', 'Id'])
print(vocabulary_df.head())

            Word  Id
0           mesc   0
1  knowledge-bas   1
2        dilemma   2
3        engaged   3
4  relationships   4


save it

In [ ]:
vocabulary_df.to_csv('/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.csv', index=False, header=False)
#remove header to keep the name of the columns

In [ ]:
# Store in a json llike a pandas dataframe
vocabulary_df.to_json('/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.json', orient='records')

In [ ]:
# Write the dictionary to the JSON file directly
with open('/content/drive/MyDrive/HM3-ADM/HW3_ADM/vocabulary.json', 'w') as jsonfile:
    json.dump(vocabulary_dict, jsonfile)

In [ ]:
vocabulary_reverse = vocabulary_df.copy()
print(vocabulary_reverse.head())

            Word  Id
0           mesc   0
1  knowledge-bas   1
2        dilemma   2
3        engaged   3
4  relationships   4


In [ ]:
tqdm.pandas()
vocabulary_reverse['reverse'] = vocabulary_reverse.Word.progress_apply(lambda item: list(processed_df.loc[processed_df.description.apply(lambda row: item in row)].index))

100%|██████████| 12545/12545 [01:49<00:00, 114.45it/s]


In [ ]:
vocabulary_reverse.head()

,Word,Id,reverse
0,mesc,0,[2131]
1,knowledge-bas,1,"[943, 4509, 4788, 4813, 5615]"
2,dilemma,2,"[181, 552, 1998, 3728, 4275, 5368]"
3,engaged,3,"[945, 3476]"
4,relationships,4,"[233, 437, 720, 1373, 1452, 1456, 1577, 2126, ..."


In [ ]:
vocabulary_reverse[vocabulary_reverse['reverse'].apply(lambda x: len(x) == 0)]

,Word,Id,reverse


#### creating a dictionary

In [ ]:
inverted_index = vocabulary_reverse.set_index('Id')['reverse'].to_dict()
count = 0
for key, value in inverted_index.items():
    if count < 5:
        print(f"Key: {key}, Value: {value}")
        count += 1
    else:
        break

Key: 0, Value: [2131]
Key: 1, Value: [943, 4509, 4788, 4813, 5615]
Key: 2, Value: [181, 552, 1998, 3728, 4275, 5368]
Key: 3, Value: [945, 3476]
Key: 4, Value: [233, 437, 720, 1373, 1452, 1456, 1577, 2126, 2764, 3004, 3903, 4376, 4386, 5526, 5641]


In [ ]:
with open('/content/drive/MyDrive/HM3-ADM/HW3_ADM/inverted_index.json', 'w') as file:
    json.dump(inverted_index, file)

##[2.1.2]

selecting only the rows we need to return as output

In [ ]:
df_query = df[['courseName','universityName','description','url']].copy()
df_query.head()

,courseName,universityName,description,url
0,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
2,Applied Computer Science and Artificial Inte...,University of Bradford,Computer science is the foundation of many exc...,https://www.findamasters.com/masters-degrees/c...
3,Applied Economics (Banking and Financial Mar...,University of Bath,From political uncertainty to finance and recr...,https://www.findamasters.com/masters-degrees/c...
4,Applied Linguistics - MSc,University of Glasgow,This Masters focuses on how linguistic researc...,https://www.findamasters.com/masters-degrees/c...


In [ ]:
df_query.shape

(5979, 4)

writing a function which extract all documents were all words are met

In [ ]:
def engine(query):
    """
    Given a query made up by multiple word it returns the documents were ALL the word are found
    """
    doc_set_indexes = []
    words_in_query = query.split()

    for word in words_in_query:
        # Stemming the word
        stemmed_word = stemmer.stem(word)

        # Check if the stemmed word exists in the 'Word' column after applying stemming
        if stemmed_word in vocabulary_reverse['Word'].apply(stemmer.stem).values:
            # Get the document set indexes for the stemmed word
            indexes_for_word = vocabulary_reverse[vocabulary_reverse['Word'].apply(lambda x: x == stemmed_word)]['reverse'].values

            # Flatten the lists in 'reverse' column
            flattened_indexes = [item for sublist in indexes_for_word for item in sublist]

            # Append the flattened document set indexes to the list
            doc_set_indexes.append(flattened_indexes)
           # print(doc_set_indexes)

        else:
            print(f"Stemmed word '{stemmed_word}' not found in vocabulary_reverse")

    # Find the intersection of all document sets
    selected_doc = list(set.intersection(*map(set, doc_set_indexes)))

    # Select rows using iloc
    selected_rows = df.iloc[selected_doc]

    return selected_rows


In [ ]:
engine('advance knowledge')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
1,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) \nInternational: £34,750 (...",MSc,1 year full time,Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
8,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2056,Engineering with Management - MSc,University of Bristol,Faculty of Engineering,Full time,The MSc in Engineering with Management is desi...,September,Please see the university website for further ...,MSc,1 year full-time,Bristol,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2069,Enhanced Radiotherapy and Oncology Practice ...,Sheffield Hallam University,Postgraduate Courses,Part time,Enhance your knowledge of specific radiotherap...,"September, January",For part-time study the fee will be calculated...,MSc,3 years part time distance learning,Sheffield,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Engineering Management (MSc),University of Gloucestershire,School of Computing and Engineering,Full time,This course will equip you with the necessary ...,September,Please see the university website for further ...,MSc,See website for details,Gloucester,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2032,Engineering Management MSc,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,The MSc Engineering Management programme will ...,"September, January",Please see the university website for further ...,MSc,1 year Full time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2034,Engineering Management MSc,Middlesex University,Faculty of Science and Technology,Full time,The career of an engineering project manager i...,October,"UK students \nFull-time students: £10,500 \nPa...",MSc,1 year full-time; 2 years part-time,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4083,M2 Research in Management and Innovation,Université Côte d’Azur,UCA International MSc Program,Full time,"Our modern industry structures, value chains a...",September,The tuition currently stands at €243 as well a...,MSc,2 years,Nice,On Campus,France,https://www.findamasters.com/masters-degrees/c...


# [2.2] Conjunctive query & Ranking score

## [2.2.1] Inverted index

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
print(processed_df.loc[2, 'description'])

['comput', 'scienc', 'foundat', 'mani', 'excit', 'cutting-edg', 'topics', 'technolog', 'applications', 'thi', 'cover', 'ai', 'big', 'data', 'cybersecurity', 'span', 'almost', 'everi', 'aspect', 'lives', 'smartphon', 'app', 'development', 'self-driv', 'assist', 'revolutionis', 'safeti', 'effici', 'transport', 'systems', 'du', 'pandemic', 'computer-rel', 'skill', 'becom', 'import', 'requir', 'work', 'remotely', 'convert', 'comput', 'scienc', 'could', 'develop', 'problem-solv', 'practic', 'skills', 'also', 'establish', 'specialist', 'knowledg', 'tool', 'techniqu', 'creat', 'intellig', 'algorithm', 'software']


In [ ]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(processed_df['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=processed_df.index, columns=tfidf.get_feature_names_out())

In [ ]:
print(tfidf_data.head())

   'all  'appli  'bench  'big  'chip  'credits   'd  'disability  'educ  \
0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
1   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
2   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
3   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
4   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   

   'entri  ...  ﻿over  ﻿prove  ﻿regist  ﻿th  ﻿the  ﻿theoret  ﻿thi  ﻿want   ﻿y  \
0     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
1     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
2     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
3     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
4     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   

    ﻿﻿  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 rows

### Controllo la lunghezza per vedere se ci sono tutte le parole: ok (?)

In [ ]:
len(tfidf_data)

5979

### Questi due codici successivi solo per vedere se ci sono elementi diversi da 0

In [ ]:
tfidf_data.loc[tfidf_data.regist > 0]

,'all,'appli,'bench,'big,'chip,'credits,'d,'disability,'educ,'entri,...,﻿over,﻿prove,﻿regist,﻿th,﻿the,﻿theoret,﻿thi,﻿want,﻿y,﻿﻿
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Costruisco un dizionario

In [ ]:
inverted_index_2 = {}
for word in tfidf_data.columns:
    # Filtraggio dei documenti in cui la parola ha un TFIDF>0: non so se vogliamo farlo seno si leva
    word_docs = tfidf_data.index[tfidf_data[word] > 0].tolist()
    word_doc_scores = [(doc, tfidf_data.loc[doc, word]) for doc in word_docs]
    inverted_index_2[word] = word_doc_scores


### Stampiamo per vedere che esce

In [ ]:
count = 0
for word, doc_scores in inverted_index_2.items():
    print(f"Word: {word}")
    print("Document Scores:")
    print(doc_scores)
    count += 1
    if count == 10:
        break

Word: 'all
Document Scores:
[(3871, 0.1849258796135223)]
Word: 'appli
Document Scores:
[(2325, 0.1722113554646612)]
Word: 'bench
Document Scores:
[(333, 0.3466489203597607)]
Word: 'big
Document Scores:
[(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)]
Word: 'chip
Document Scores:
[(1902, 0.1992739368962018)]
Word: 'credits
Document Scores:
[(2317, 0.18303193893688977), (5926, 0.08382342995897132)]
Word: 'd
Document Scores:
[(2160, 0.194367236072988), (4863, 0.14435725548964343)]
Word: 'disability
Document Scores:
[(4783, 0.17222181095019545)]
Word: 'educ
Document Scores:
[(2618, 0.2647135148766833)]
Word: 'entri
Document Scores:
[(641, 0.20952396586542038)]


## Ma noi dobbiamo avere l'id della parola come chiave non la parola stessa e questo id lo prendiamo da vocabulary e creiamo un nuovo dizionario che è quello richiesto dalla traccia

In [ ]:
new_inverted_index = {}
for word, doc_scores in inverted_index_2.items():
    word_id = vocabulary_df[vocabulary_df['Word'] == word]['Id'].values
    if len(word_id) > 0:
        word_id = word_id[0]
        new_inverted_index[word_id] = doc_scores


### Stampiamo

In [ ]:
count = 0
for word, doc_scores in new_inverted_index.items():
    print(f"Word: {word}")
    print("Document Scores:")
    print(doc_scores)
    count += 1
    if count == 10:
        break

Word: 7237
Document Scores:
[(3871, 0.1849258796135223)]
Word: 8785
Document Scores:
[(2325, 0.1722113554646612)]
Word: 9700
Document Scores:
[(333, 0.3466489203597607)]
Word: 3365
Document Scores:
[(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)]
Word: 1201
Document Scores:
[(1902, 0.1992739368962018)]
Word: 650
Document Scores:
[(2317, 0.18303193893688977), (5926, 0.08382342995897132)]
Word: 11512
Document Scores:
[(2160, 0.194367236072988), (4863, 0.14435725548964343)]
Word: 7980
Document Scores:
[(4783, 0.17222181095019545)]
Word: 2565
Document Scores:
[(2618, 0.2647135148766833)]
Word: 4168
Document Scores:
[(641, 0.20952396586542038)]


### Stampiamo le prime 10 chiavi per vedere se è come è richiesto

In [ ]:
# Stampa solo le prime n chiavi del nuovo dizionario
n = 10  # Imposta il numero di chiavi da stampare

# Stampare le prime n chiavi con i relativi valori
partial_inverted_index = {k: new_inverted_index[k] for k in list(new_inverted_index.keys())[:n]}
print(partial_inverted_index)

{7237: [(3871, 0.1849258796135223)], 8785: [(2325, 0.1722113554646612)], 9700: [(333, 0.3466489203597607)], 3365: [(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)], 1201: [(1902, 0.1992739368962018)], 650: [(2317, 0.18303193893688977), (5926, 0.08382342995897132)], 11512: [(2160, 0.194367236072988), (4863, 0.14435725548964343)], 7980: [(4783, 0.17222181095019545)], 2565: [(2618, 0.2647135148766833)], 4168: [(641, 0.20952396586542038)]}


In [ ]:
key_to_print = 0
if key_to_print in new_inverted_index:
    print(new_inverted_index[key_to_print])
else:
    print("La chiave specificata non è presente nel dizionario.")

[(2131, 0.29021714769031837)]


In [ ]:
key_to_print = 0
if key_to_print in inverted_index:
    print(inverted_index[key_to_print])
else:
    print("La chiave specificata non è presente nel dizionario.")

[2131]


In [ ]:
target_id = 0
word = vocabulary_df.loc[vocabulary_df['Id'] == target_id, 'Word'].values[0]
print(word)

mesc


In [ ]:
target_index=2131  # Indice desiderato
# Verifica se l'indice è presente nel DataFrame
if target_index in processed_df.index:
    description_value = processed_df.loc[target_index, 'description']
    print(description_value)
else:
    print(f"L'indice {target_index} non è presente nel DataFrame.")

['master', 'environment', 'social', 'chang', 'mesc', 'program', 'interdisciplinari', 'research-intens', 'program', 'focusing', 'train', 'relat', 'human', 'roles', 'environment', 'offers', 'unique', 'gradu', 'train', 'experience', 'blend', 'appli', 'analyt', 'techniques', 'critic', 'thinking', 'best', 'practices', 'sustainable', 'ecosystems', 'policies', 'societies']


####cosine similarity

In [ ]:
a = engine("air pollution")

In [ ]:
a

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2150,Environmental Engineering - MSc,Imperial College London,Civil and Environmental Engineering,Full time,This course provides training in all aspects o...,October,Please see the university website for further ...,MSc,1 year full-time,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2216,Environmental Pollution Control MSc,University of Southampton,Faculty of Environmental and Life Sciences,Full time,Explore Environmental Pollution Control at a g...,September,Please see the university website for further ...,MSc,1 Year,Southampton,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2128,Environmental Analytical Chemistry - MSc,University of Bristol,Faculty of Science,Full time,"Climate change, and air, water and soil pollut...",September,Please see the university website for further ...,MSc PGDip PGCert,One year full-time; two years part-time,Bristol,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
5430,Air Pollution Management and Control - MSc/P...,University of Birmingham,"School of Geography, Earth and Environmental S...",Full time,Our Air Pollution Management and Control MSc i...,September,Please see the university website for further ...,PGDip MSc,"1 year full time, 2 years part time",Birmingham,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
a.index

Int64Index([0, 2150, 2216, 2128, 5430], dtype='int64')

In [ ]:
df_query_processed= processed_df[['courseName','universityName','description','url']].copy()

In [ ]:
df_query_processed.head()

,courseName,universityName,description,url
0,air quality solutions msc,university leeds,"[7, million, peopl, estim, die, everi, year, d...",https //www.findamasters.com/masters-degrees/c...
1,analytical toxicology msc,king college london,"[analyt, toxicolog, msc, uniqu, studi, cours, ...",https //www.findamasters.com/masters-degrees/c...
2,applied computer science artificial intelligen...,university bradford,"[comput, scienc, foundat, mani, excit, cutting...",https //www.findamasters.com/masters-degrees/c...
3,applied economics banking financial markets on...,university bath,"[polit, uncertainti, financ, recruit, demands,...",https //www.findamasters.com/masters-degrees/c...
4,applied linguistics msc,university glasgow,"[thi, master, focus, linguist, research, help,...",https //www.findamasters.com/masters-degrees/c...


In [ ]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(df_query_processed['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=df_query_processed.index, columns=tfidf.get_feature_names_out())

In [ ]:
print(tfidf_data.head())

   'all  'appli  'bench  'big  'chip  'credits   'd  'disability  'educ  \
0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
1   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
2   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
3   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
4   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   

   'entri  ...  ﻿over  ﻿prove  ﻿regist  ﻿th  ﻿the  ﻿theoret  ﻿thi  ﻿want   ﻿y  \
0     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
1     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
2     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
3     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
4     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   

    ﻿﻿  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 rows

I need to take the cosine similary for the words in the query.

In [ ]:
def rank_documents(query1):
    """
    Given a query, computes the tfidf for that query and evaluate the cosine similarity con for query given
    the document extracted with engine function. Returns the top-k documents
    """
    ## COMPUTING THE TFIDF FOR THE QUERY

   # Tokenize the query and save the stammed words for computinf tfidf
    tokens = word_tokenize(query1)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    tfidf_vectorizer = TfidfVectorizer()

    # Combine the stemmed tokens considering it single string (document)
    stemmed_query = ' '.join(stemmed_tokens)

    # Fit the vectorizer on the list of queries and transform the query into a tfidf vector
    query_tfidf = tfidf_vectorizer.fit_transform([stemmed_query])

    # Convert the TF-IDF matrix to a dense pandas DataFrame
    tfidf_query = pd.DataFrame(query_tfidf.todense(), index=['query'], columns=tfidf_vectorizer.get_feature_names_out())

    ## OBTAINE THE TFIDF BEWTEEN THE QUERY AND THE DOCUMENT
    # Initialize a heap for the k-top results
    k_top_results = []

    # Iterate over the documents
    for idx, document_text in enumerate(engine(query1)['description']):

        # Tokenize and preprocess the document text
        #document_tokens = [stemmer.stem(word) for word in word_tokenize(document_text)]
        #document_text_processed = ' '.join(document_tokens)

        # Transform the document text using the fitted vectorizer
        #tfidf_document = tfidf_vectorizer.transform([document_text_processed])

        for index, tfidf in inverted_index_2['air']:
            if index in engine('air pollution')['description'].index:
                print('nel documento',{index}, 'la cosine similarity e',tfidf)
        # Compute cosine similarity between the query and the document
        similarity_score = cosine_similarity(tfidf_query, tfidf_document)[0, 0]

       ## USING A HEAP TO HAVE THE TOP-K DOCUMENTS
        # Append the similarity score and document index to the k_top_results list
        k_top_results.append((round(similarity_score,5), engine(query1)['description'].index[idx]))


    # Retrieve the k-top results from the heap using nlargest
    k_top_results = heapq.nlargest(10, k_top_results, key=lambda x: x[0])
    rank_df = pd.DataFrame()

    for cossim, indexsorted in k_top_results:
        # Return the information about the document sorted by cosine similarity
            document_info = df_query.loc[[indexsorted]].copy()
#            return document_info
            document_info['Similarity'] = cossim
            rank_df = pd.concat([rank_df, document_info])

    # Eventually we could reset the index of the final DataFrame
    # rank_df = rank_df.reset_index(drop=True)

    return rank_df

In [ ]:
rank_documents("advanced knowledge")

,courseName,universityName,description,url,Similarity
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...,1.0
8,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,1.0
2056,Engineering with Management - MSc,University of Bristol,The MSc in Engineering with Management is desi...,https://www.findamasters.com/masters-degrees/c...,1.0
2073,Enterprise Management MSc/PGDip,University College Birmingham,In today’s increasingly competitive business e...,https://www.findamasters.com/masters-degrees/c...,1.0
2089,Entrepreneurship and Innovation Management MSc,University of Exeter,Overview\nGain a fully transferable and implem...,https://www.findamasters.com/masters-degrees/c...,1.0
2044,Engineering Materials (Advanced Mechanical E...,University of Southampton,Develop specialist knowledge of mechanical eng...,https://www.findamasters.com/masters-degrees/c...,1.0
4169,Accounting and Finance - MSc,University of Sussex,On this MSc you’ll advance your accounting and...,https://www.findamasters.com/masters-degrees/c...,1.0
4180,Management and Digital Business (with Advanc...,Liverpool John Moores University,This Advanced Practice course provides an in-d...,https://www.findamasters.com/masters-degrees/c...,1.0
4203,Management in Construction (MSc),Kingston University,This industry-accredited course is suitable fo...,https://www.findamasters.com/masters-degrees/c...,1.0
115,Bioscience (MSc),KAUST,The Bioscience (B) program plays a key role in...,https://www.findamasters.com/masters-degrees/c...,1.0


In [ ]:
rank_documents("air pollution")

,courseName,universityName,description,url,Similarity
5430,Air Pollution Management and Control - MSc/P...,University of Birmingham,Our Air Pollution Management and Control MSc i...,https://www.findamasters.com/masters-degrees/c...,0.99589
0,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...,0.98058
2128,Environmental Analytical Chemistry - MSc,University of Bristol,"Climate change, and air, water and soil pollut...",https://www.findamasters.com/masters-degrees/c...,0.98058
2150,Environmental Engineering - MSc,Imperial College London,This course provides training in all aspects o...,https://www.findamasters.com/masters-degrees/c...,0.94868
2216,Environmental Pollution Control MSc,University of Southampton,Explore Environmental Pollution Control at a g...,https://www.findamasters.com/masters-degrees/c...,0.83205


In [ ]:
rank_documents("sustainability measures")

,courseName,universityName,description,url,Similarity
2794,Geospatial & Mapping Sciences - MSc/PgDip/Pg...,University of Glasgow,This MSc in Geospatial & Mapping Sciences focu...,https://www.findamasters.com/masters-degrees/c...,1.0
452,Civil and Environmental Engineering (MSc),University of Brescia,The MSc trains graduates who have in-depth kno...,https://www.findamasters.com/masters-degrees/c...,1.0
2814,Global Business Management MSc,Università Cattolica del Sacro Cuore,The Master of Science in Global Business Manag...,https://www.findamasters.com/masters-degrees/c...,1.0


In [ ]:
def rank_documents(query1):
    """
    Given a query, computes the tfidf for that query and evaluate the cosine similarity con for query given
    the document extracted with engine function. Returns the top-k documents
    """
    ## COMPUTING THE TFIDF FOR THE QUERY

    # Tokenize the query and save the stemmed words for computing tfidf
    tokens = word_tokenize(query1)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    ## OBTAINE THE TFIDF BEWTEEN THE QUERY AND THE DOCUMENT
    # Initialize a heap for the k-top results
    k_top_results = []

    # Iterate over the documents
    for idx, document_text in enumerate(engine(query1)['description']):
        # Initialize the cosine similarity score
        similarity_score = 0

        # Iterate over the query words and their TFIDF values in the document
        for word in stemmed_tokens:
            # Check if the word is in the inverted index
            if word in inverted_index_2 and idx in inverted_index_2[word]:
                # Retrieve the TFIDF value from the inverted index
                tfidf = inverted_index_2[word][idx]
                # Accumulate the TFIDF values for each word
                similarity_score += tfidf

        # Append the similarity score and document index to the k_top_results list
        k_top_results.append((round(similarity_score, 5), engine(query1)['description'].index[idx]))


    # Retrieve the k-top results from the heap using nlargest
    k_top_results = heapq.nlargest(10, k_top_results, key=lambda x: x[0])
    rank_df = pd.DataFrame()

    for cossim, indexsorted in k_top_results:
        # Return the information about the document sorted by cosine similarity
            document_info = df_query.loc[[indexsorted]].copy()
#            return document_info
            document_info['Similarity'] = cossim
            rank_df = pd.concat([rank_df, document_info])

    # Eventually we could reset the index of the final DataFrame
    # rank_df = rank_df.reset_index(drop=True)

    return rank_df

In [ ]:
rank_documents("advanced knowledge") #4m36sec

,courseName,universityName,description,url,Similarity
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...,0
5,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...,0
8,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,0
2056,Engineering with Management - MSc,University of Bristol,The MSc in Engineering with Management is desi...,https://www.findamasters.com/masters-degrees/c...,0
2069,Enhanced Radiotherapy and Oncology Practice ...,Sheffield Hallam University,Enhance your knowledge of specific radiotherap...,https://www.findamasters.com/masters-degrees/c...,0
25,"Healthcare Technologies MSc, MRes",King’s College London,The Healthcare Technologies MSc/MRes will trai...,https://www.findamasters.com/masters-degrees/c...,0
2073,Enterprise Management MSc/PGDip,University College Birmingham,In today’s increasingly competitive business e...,https://www.findamasters.com/masters-degrees/c...,0
2089,Entrepreneurship and Innovation Management MSc,University of Exeter,Overview\nGain a fully transferable and implem...,https://www.findamasters.com/masters-degrees/c...,0
2044,Engineering Materials (Advanced Mechanical E...,University of Southampton,Develop specialist knowledge of mechanical eng...,https://www.findamasters.com/masters-degrees/c...,0
49,Biomedical Engineering with Data Analytics MSc,"City, University of London",Key information\nThis course provides a compre...,https://www.findamasters.com/masters-degrees/c...,0


In [ ]:
def rank_documents(query1):
    """
    Given a query, computes the tfidf for that query and evaluates the cosine similarity for the query given
    the document extracted with the engine function. Returns the top-k documents.
    """
    # Tokenize the query and save the stemmed words for computing tfidf
    tokens = word_tokenize(query1)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    tfidf_vectorizer = TfidfVectorizer()

    # Combine the stemmed tokens considering it single string (document)
    stemmed_query = ' '.join(stemmed_tokens)

    # Fit the vectorizer on the list of queries and transform the query into a tfidf vector
    query_tfidf = tfidf_vectorizer.fit_transform([stemmed_query])


    # Initialize a heap for the k-top results
    k_top_results = []

    # Iterate over the documents
    for idx, document_text in enumerate(engine(query1)['description']):
        # Initialize the cosine similarity score
        similarity_score = 0

        # Iterate over the query words and their TFIDF values in the document
        for word in stemmed_tokens:
            # Check if the word is in the inverted index
            if word in inverted_index_2 and idx in inverted_index_2[word]:
                # Retrieve the TFIDF value from the inverted index
                tfidf = inverted_index_2[word][idx]
                # Accumulate the dot product of TFIDF vectors for each word
                similarity_score += tfidf

          # Convert the query TFIDF to a dense matrix
        query_tfidf_dense = query_tfidf.todense()


        # Compute cosine similarity between the query and the document
        similarity_score = cosine_similarity(tfidf_query, tfidf_vectorizer.transform([document_text]))[0, 0]

        # Append the similarity score and document index to the k_top_results list
        k_top_results.append((round(similarity_score, 5), engine(query1)['description'].index[idx]))

    # Retrieve the k-top results from the heap using nlargest
    k_top_results = heapq.nlargest(10, k_top_results, key=lambda x: x[0])
    rank_df = pd.DataFrame()

    for cossim, indexsorted in k_top_results:
        # Return the information about the document sorted by cosine similarity
        document_info = df_query.loc[[indexsorted]].copy()
        document_info['Similarity'] = cossim
        rank_df = pd.concat([rank_df, document_info])

    # Eventually, we could reset the index of the final DataFrame
    # rank_df = rank_df.reset_index(drop=True)

    return rank_df


#[7] Algorithmic question

The problem involves helping Leonardo create a fake report on the number of hours he worked each day for the past d days. Leonardo has the total sum of hours he worked, the number of days, and HR limitations for each day.

## Implementation of the code

In [ ]:
# Input the total number of days and the total number of hours
total_days, sum_hours = map(int, input().split())

# Initialize a list to store the ranges of hours for each day
day_min_max = []

# Input the minimum and maximum hours for each day and create a list of ranges
for _ in range(total_days):
    min1, max1 = map(int, input().split())
    day_min_max.append([i for i in range(min1, max1 + 1)])

def find_combinations_with_sum(nested_list, target_sum):
    """
    Having a nested list with the information of the overall amount of days and hour
    worked and informationa bout mintime e maxtime for a single day, returns wether
    it it possibile to return a fake report which respect costrains
    """
    # Get the minimum and maximum hours for each day
    min_hours = [day[0] for day in day_min_max]
    max_hours = [day[-1] for day in day_min_max]

    # Check if the total hours are within the valid range
    if sum_hours >= sum(min_hours) and sum_hours <= sum(max_hours):
        print('YES')
    else:
        print('NO')

    # Iterate through all combinations of hours and print those with the target sum
    for combination in product(*nested_list):
        if sum(combination) == target_sum:
            print(*combination)

# Call the function with the provided day_min_max and sum_hours
find_combinations_with_sum(day_min_max, sum_hours)


1 1
2 3
NO



## Let's combine the time complexity of individual parts to determine the overall time complexity of the provided code:



Initializing min_hours and max_hours: O(total_days)
* Checking total hours range: O(1)
* Generating combinations: O(product of sizes of nested_list)
* Overall, the dominant factor is the third part involving the generation of combinations. Therefore, the overall time complexity is O(product of sizes of nested_list).

#### example : if we have only two working days


let's calculate the number of combinations generated by the product function for two days, each with a range of (max - min + 1) hours

day1 = max_hour - min_hour + 1 = n1

day2 = max_hour - min_hour + 1 = n2


Complexity would be : O(n1 * n2 )

## ChatGpt Implementation



both awsers are right but we think the complexity of the code that Chatgpt has been generated is better since it use backtracking algorithm which avoids generation all combination

##  **Optimality**:


The solution can be considered suboptimal in terms of time complexity, especially when the ranges for each day are large.
It generates all combinations using itertools.product, which can be inefficient for large inputs.
An improvement could be to use a backtracking algorithm that explores valid combinations more efficiently, terminating the search as soon as a valid combination is found.